In [1]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [2]:
browser = webdriver.Chrome("./chromedriver")
browser.get("https://www.linkedin.com")

In [3]:
username = browser.find_element_by_id("session_key")
username.send_keys("mjeng@ucsc.edu")
password = browser.find_element_by_id("session_password")
password.send_keys("CSE115Project")

In [4]:
login_button=browser.find_element_by_class_name("sign-in-form__submit-button")
login_button.click()

In [5]:
browser.get("https://www.linkedin.com/jobs")

In [9]:
job_search_bar = browser.find_element_by_id("jobs-search-box-keyword-id-ember37")
job_search_bar.send_keys("Software Engineer")

In [10]:
job_search_button = browser.find_element_by_class_name("jobs-search-box__submit-button")
job_search_button.click()

In [38]:
# ------------------
# parsing the visible webpage
driver = browser
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
#job_container = lxml_soup.find('ul', class_ = 'jobs-search-results__list')
job_container = lxml_soup.find_all('li', class_ = 'jobs-search-results__list-item')

#print(job_container[0])
print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 25 jobs.


In [40]:
job_container[0]

<li class="jobs-search-results__list-item occludable-update p0 relative ember-view" data-occludable-entity-urn="urn:li:fs_normalized_jobPosting:2484387372" id="ember600">
<div>
<div class="job-card-container relative job-card-list job-card-container--clickable job-card-list--underline-title-on-hover jobs-search-results-list__list-item--active jobs-search-two-pane__job-card-container--viewport-tracking-0" data-job-id="2484387372">
<div class="flex-grow-1 artdeco-entity-lockup artdeco-entity-lockup--size-4 ember-view" id="ember601">
<div class="mr1 artdeco-entity-lockup__image artdeco-entity-lockup__image--type-square ember-view" id="ember602" type="square">
<a class="disabled ember-view job-card-container__link" data-control-id="tWaEXnB/ZLeCGdbT/5rUyA==" href="/jobs/view/2484387372/?eBP=CwEAAAF4yChNQ32M8O4xXqekJ4gf1JTMFFVj6niNVr_ekXX31mhUTImYjsp-WGZ65uyx_dgx-XZeMqVqxEQCrD-Eu4Bzr4yQlyr1P3ZTLAL6MDmrCe1sKtqiAiCdX_0p0NQqnalcZk2ChJufAcEdePl4N-Pq1Mz-YUi31nXqaFNJJ17EVz9xw37QnKZaaLc2T-UprWb9wnW

In [39]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find(class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)
    
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
    
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
    industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    industries.append(industry_type)
    
    x = x+1

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))